## Import Libraries

In [1]:
from fastai.vision.all import * 
import params 

import wandb

In [7]:
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /home/moro/.netrc


True

## Download Dataset

In [2]:
## lets set the url 
url = 'https://storage.googleapis.com/wandb_course/bdd_simple_1k.zip'

path = Path(untar_data(url, force_download=True))

In [3]:
path.ls()

(#3) [Path('/home/moro/.fastai/data/bdd_simple_1k/LICENSE.txt'),Path('/home/moro/.fastai/data/bdd_simple_1k/labels'),Path('/home/moro/.fastai/data/bdd_simple_1k/images')]

## Exploratory Data Analysis

In [4]:
def label_func(fname):
    return (fname.parent.parent/"labels")/f"{fname.stem}_mask.png"

def get_classes_per_image(mask_data, class_labels):
    unique = list(np.unique(mask_data))
    result_dict = {}
    for _class in class_labels.keys():
        result_dict[class_labels[_class]] = int(_class in unique)
    return result_dict

def _create_table(image_files, class_labels):
    "Create a table with the dataset"
    labels = [str(class_labels[_lab]) for _lab in list(class_labels)]
    table = wandb.Table(columns=["File_Name", "Images", "Split"] + labels)
    
    for i, image_file in progress_bar(enumerate(image_files), total=len(image_files)):
        image = Image.open(image_file)
        mask_data = np.array(Image.open(label_func(image_file)))
        class_in_image = get_classes_per_image(mask_data, class_labels)
        table.add_data(
            str(image_file.name),
            wandb.Image(
                    image,
                    masks={
                        "predictions": {
                            "mask_data": mask_data,
                            "class_labels": class_labels,
                        }
                    }
            ),
            "None", # we don't have a dataset split yet
            *[class_in_image[_lab] for _lab in labels]
        )
    
    return table

In [8]:
## lets start a new WANDB run..
run = wandb.init(project=params.WANDB_PROJECT, entity=params.ENTITY, job_type="upload")


wandb: Currently logged in as: moro23. Use `wandb login --relogin` to force relogin


In [10]:
## lets create an artifact
artifact = wandb.Artifact(params.RAW_DATA_AT, type="raw_data")

Exception in thread SystemMonitor:
Traceback (most recent call last):
  File "/usr/lib64/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib64/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/home/moro/.local/share/virtualenvs/ml_object_detection-upmsBY-g/lib/python3.10/site-packages/wandb/sdk/internal/system/system_monitor.py", line 118, in _start
    asset.start()
  File "/home/moro/.local/share/virtualenvs/ml_object_detection-upmsBY-g/lib/python3.10/site-packages/wandb/sdk/internal/system/assets/cpu.py", line 166, in start
    self.metrics_monitor.start()
  File "/home/moro/.local/share/virtualenvs/ml_object_detection-upmsBY-g/lib/python3.10/site-packages/wandb/sdk/internal/system/assets/interfaces.py", line 168, in start
    logger.info(f"Started {self._process.name}")
AttributeError: 'NoneType' object has no attribute 'name'


In [ ]:
## lets add a file to artifact
artifact.add_file(path/'LICENSE.txt', name='LICENSE.txt')

In [11]:
## lets add folders to artifact
artifact.add_dir(path/'images', name='images')
artifact.add_dir(path/'labels', name='labels')

wandb: Adding directory to artifact (/home/moro/.fastai/data/bdd_simple_1k/images)... Done. 26.4s
wandb: Adding directory to artifact (/home/moro/.fastai/data/bdd_simple_1k/labels)... Done. 26.0s


In [12]:
image_files = get_image_files(path/'images', recurse=False)


In [13]:
table = _create_table(image_files, params.BDD_CLASSES)

In [14]:
## lets add table to artifact
artifact.add(table, 'eda_table')

ArtifactManifestEntry(path='eda_table.table.json', digest='2a+QC661MlcQ7H3Bcy+B8Q==', ref=None, birth_artifact_id=None, size=588824, extra={}, local_path='/home/moro/.local/share/wandb/artifacts/staging/tmp76bw1umk')

In [15]:
## lets log artifact
run.log_artifact(artifact)

In [16]:
## lets finish the run
run.finish()